<h1> Exploring tf.transform </h1>

While Pandas is fine for experimenting, for operationalization of your workflow, it is better to do preprocessing in Apache Beam. This will also help if you need to preprocess data in flight, since Apache Beam also allows for streaming.

Only specific combinations of TensorFlow/Beam are supported by tf.transform. So make sure to get a combo that is.

* TFT 0.6.0
* TF 1.6 or higher
* Apache Beam [GCP] 2.4.0 or higher

In [1]:
%bash
pip uninstall -y google-cloud-dataflow
pip install --upgrade --force tensorflow_transform==0.6.0 apache-beam[gcp]

Uninstalling google-cloud-dataflow-2.0.0:
  Successfully uninstalled google-cloud-dataflow-2.0.0
  Running setup.py bdist_wheel for tensorflow-transform: started
  Running setup.py bdist_wheel for tensorflow-transform: finished with status 'done'
  Stored in directory: /content/.cache/pip/wheels/67/67/f9/a5f81a500bbbddf2822eb4371681ea21f6ca4ac16d94e4fcdf
  Running setup.py bdist_wheel for httplib2: started
  Running setup.py bdist_wheel for httplib2: finished with status 'done'
  Stored in directory: /content/.cache/pip/wheels/36/f2/49/5adbf90fba31e02a7784e1147d7f8b6c4af3718739e568c8cb
  Running setup.py bdist_wheel for dill: started
  Running setup.py bdist_wheel for dill: finished with status 'done'
  Stored in directory: /content/.cache/pip/wheels/17/74/0a/250b442a5196d5c0d80f7d4259a0a26a9c9af0e6b0ee98a335
  Running setup.py bdist_wheel for pyvcf: started
  Running setup.py bdist_wheel for pyvcf: finished with status 'done'
  Stored in directory: /content/.cache/pip/wheels/81/91/41/

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    DEPRECATION: Uninstalling a distutils installed project (crcmod) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only partially uninstall the project.
    DEPRECATION: Uninstalling a distutils installed project (pyyaml) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only partially uninstall the project.
    DEPRECATION: Uninstalling a distutils installed project (certifi) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only partially uninstall the project.
    DEPRECATION: Uninstalling a distutils installed project (future) has been deprecated and will be removed in a future version. This is due to the 

<b>Restart the kernel</b> after you do a pip install (click on the <b>Reset</b> button in Datalab)

In [1]:
%bash
pip freeze | grep -e 'flow\|beam'

apache-airflow==1.9.0
apache-beam==2.4.0
tensorflow==1.8.0
tensorflow-transform==0.6.0


You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import tensorflow as tf
import tensorflow_transform as tft
import shutil
print(tf.__version__)

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


In [2]:
# change these to try this notebook out
BUCKET = 'eim-muse'
PROJECT = 'eim-muse'
REGION = 'us-central1'

In [65]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:286: DeprecationWarning: Interpreting naive datetime as local 2018-06-13 17:12:53.337004. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [4]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Input source: BigQuery

Get data from BigQuery but defer filtering etc. to Beam.
Note that the dayofweek column is now strings.

In [5]:
import google.datalab.bigquery as bq
def create_query(phase, EVERY_N):
  """
  phase: 1=train 2=valid
  """
  base_query = """
SELECT *
FROM
  `eim-muse.hallelujah_effect.full_hallelujah_trials_cleaned`
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} WHERE MOD(FARM_FINGERPRINT(id), 10) < 7".format(base_query)
    else:
      # Validation
      query = "{0} WHERE MOD(FARM_FINGERPRINT(id), 10) >= 8".format(base_query)
  else:
      query = "{0} WHERE MOD(FARM_FINGERPRINT(id), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, None)

In [6]:
df_valid = bq.Query(query).execute().result().to_dataframe()
df_valid.head()
df_valid.describe()

,age,most_engaged,most_enjoyed,artistic,fault,imagination,lazy,nervous,outgoing,reserved,stress,thorough,trusting,activity,engagement,familiarity,like_dislike,positivity,tension,terminal
count,34.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000
mean,23.411765,2.647059,2.352941,2.352941,2.882353,3.705882,3.470588,3.058824,3.529412,2.882353,3.352941,3.294118,3.941176,3.235294,2.911765,3.441176,2.911765,3.235294,3.764706,2.735294
std,11.760534,1.114741,1.114741,1.169464,1.053705,1.212678,1.124591,1.144038,0.717430,0.927520,0.931476,0.985184,0.658653,0.854891,1.564137,1.501336,1.504894,1.207522,1.074747,1.081772
min,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,17.250000,2.000000,1.000000,1.000000,2.000000,3.000000,3.000000,2.000000,3.000000,2.000000,3.000000,3.000000,4.000000,3.000000,1.000000,2.250000,1.000000,2.250000,3.000000,2.000000
50%,21.000000,3.000000,2.000000,2.000000,3.000000,4.000000,4.000000,3.000000,4.000000,3.000000,3.000000,3.000000,4.000000,3.000000,3.000000,4.000000,3.000000,3.500000,4.000000,3.000000
75%,31.750000,3.000000,3.000000,3.000000,4.000000,5.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,4.000000
max,56.000000,4.000000,4.000000,4.000000,4.000000,5.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,5.000000,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.000000


## Create ML dataset using tf.transform and Dataflow

Let's use Cloud Dataflow to read in the BigQuery data and write it out as CSV files. Along the way, let's use tf.transform to do scaling and transforming. Using tf.transform allows us to save the metadata to ensure that the appropriate transformations get carried out during prediction as well.

In [7]:
%writefile requirements.txt
tensorflow-transform==0.6.0

Overwriting requirements.txt


In [52]:
import datetime
import tensorflow as tf
import apache_beam as beam
import tensorflow_transform as tft
from tensorflow_transform.beam import impl as beam_impl

def is_valid(inputs):
#   try:
#     pickup_longitude = inputs['pickuplon']
#     dropoff_longitude = inputs['dropofflon']
#     pickup_latitude = inputs['pickuplat']
#     dropoff_latitude = inputs['dropofflat']
#     hourofday = inputs['hourofday']
#     dayofweek = inputs['dayofweek']
#     passenger_count = inputs['passengers']
#     fare_amount = inputs['fare_amount']
#     return (fare_amount >= 2.5 and pickup_longitude > -78 and pickup_longitude < -70 \
#       and dropoff_longitude > -78 and dropoff_longitude < -70 and pickup_latitude > 37 \
#       and pickup_latitude < 45 and dropoff_latitude > 37 and dropoff_latitude < 45 \
#       and passenger_count > 0)
#   except:
#     return False
  try:
    return True
  except:
    return False
  
def preprocess_tft(inputs):
      print(inputs)
#       import datetime
#       print inputs
#       result = {}
#       result['fare_amount'] = tf.identity(inputs['fare_amount'])     
#       result['dayofweek'] = tft.string_to_int(inputs['dayofweek']) # builds a vocabulary
#       result['hourofday'] = tf.identity(inputs['hourofday']) # pass through
#       result['pickuplon'] = (tft.scale_to_0_1(inputs['pickuplon'])) # scaling numeric values
#       result['pickuplat'] = (tft.scale_to_0_1(inputs['pickuplat']))
#       result['dropofflon'] = (tft.scale_to_0_1(inputs['dropofflon']))
#       result['dropofflat'] = (tft.scale_to_0_1(inputs['dropofflat']))
#       result['passengers'] = tf.cast(inputs['passengers'], tf.float32) # a cast
#       result['key'] = tf.as_string(tf.ones_like(inputs['passengers'])) # arbitrary TF func
#       # engineered features
#       latdiff = inputs['pickuplat'] - inputs['dropofflat']
#       londiff = inputs['pickuplon'] - inputs['dropofflon']
#       result['latdiff'] = tft.scale_to_0_1(latdiff)
#       result['londiff'] = tft.scale_to_0_1(londiff)
#       dist = tf.sqrt(latdiff * latdiff + londiff * londiff)
#       result['euclidean'] = dist
#       return result
      import datetime
      print inputs
      result = {}
      result['age'] = tft.scale_to_0_1(inputs['age'])
      result['activity'] = tft.scale_to_0_1(inputs['activity'])
      result['hallelujah_reaction'] = tf.cast(inputs['hallelujah_reaction'], tf.int64)
#       result['concentration'] = tft.scale_to_0_1(inputs['concentration'])
      result['hearing_impairments'] = tf.cast(inputs['hearing_impairments'], tf.int64)
      result['nationality'] = tf.identity(inputs['nationality'])
      result['engagement'] = tft.scale_to_0_1(inputs['engagement'])
      result['familiarity'] = tft.scale_to_0_1(inputs['familiarity'])
      result['like_dislike'] = tft.scale_to_0_1(inputs['like_dislike'])
      result['positivity'] = tft.scale_to_0_1(inputs['positivity'])
      result['tension'] = tft.scale_to_0_1(inputs['tension'])
      result['sex'] = tf.identity(inputs['sex'])
      result['location'] = tf.identity(inputs['location'])
      result['language'] = tf.identity(inputs['language'])
#       result['dayofweek'] = tft.string_to_int(inputs['dayofweek']) # builds a vocabulary
#       result['hourofday'] = tf.identity(inputs['hourofday']) # pass through
#       result['pickuplon'] = (tft.scale_to_0_1(inputs['pickuplon'])) # scaling numeric values
#       result['pickuplat'] = (tft.scale_to_0_1(inputs['pickuplat']))
#       result['dropofflon'] = (tft.scale_to_0_1(inputs['dropofflon']))
#       result['dropofflat'] = (tft.scale_to_0_1(inputs['dropofflat']))
#       result['passengers'] = tf.cast(inputs['passengers'], tf.float32) # a cast
#       result['key'] = tf.as_string(tf.ones_like(inputs['passengers'])) # arbitrary TF func
#       # engineered features
#       latdiff = inputs['pickuplat'] - inputs['dropofflat']
#       londiff = inputs['pickuplon'] - inputs['dropofflon']
#       result['latdiff'] = tft.scale_to_0_1(latdiff)
#       result['londiff'] = tft.scale_to_0_1(londiff)
#       dist = tf.sqrt(latdiff * latdiff + londiff * londiff)
#       result['euclidean'] = dist
      return result

def preprocess(in_test_mode):
  import os
  import os.path
  import tempfile
  from apache_beam.io import tfrecordio
  from tensorflow_transform.coders import example_proto_coder
  from tensorflow_transform.tf_metadata import dataset_metadata
  from tensorflow_transform.tf_metadata import dataset_schema
  from tensorflow_transform.beam import tft_beam_io
  from tensorflow_transform.beam.tft_beam_io import transform_fn_io

  job_name = 'hallelujah-effect-features' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')    
  if in_test_mode:
    import shutil
    print 'Launching local job ... hang on'
    OUTPUT_DIR = './preproc_tft'
    shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
    
    # This could be an issue
    EVERY_N = 5
    
  else:
    print 'Launching Dataflow job {} ... hang on'.format(job_name)
    OUTPUT_DIR = 'gs://{0}/analysis/hallelujah-effect/preproc_tft/'.format(BUCKET)
    import subprocess
    subprocess.call('gsutil rm -r {}'.format(OUTPUT_DIR).split())
    
    # This could be an issue
    EVERY_N = 5
    
  # None of this EVERY_N business
  EVERY_N = None
    
  options = {
    'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
    'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
    'job_name': job_name,
    'project': PROJECT,
    'max_num_workers': 24,
    'teardown_policy': 'TEARDOWN_ALWAYS',
    'no_save_main_session': True,
    'requirements_file': 'requirements.txt'
  }
  opts = beam.pipeline.PipelineOptions(flags=[], **options)
  if in_test_mode:
    RUNNER = 'DirectRunner'
  else:
    RUNNER = 'DataflowRunner'

  # set up metadata
  raw_data_schema = {
    colname : dataset_schema.ColumnSchema(tf.string, [], dataset_schema.FixedColumnRepresentation())
                   for colname in 'nationality,sex,location,language'.split(',')
  }
  raw_data_schema.update({
      colname : dataset_schema.ColumnSchema(tf.float32, [], dataset_schema.FixedColumnRepresentation())
                   for colname in 'age,activity,engagement,familiarity,like_dislike,positivity,tension'.split(',')
    })
#   raw_data_schema.update({
#       colname : dataset_schema.ColumnSchema(tf.int64, [], dataset_schema.FixedColumnRepresentation())
#                    for colname in 'activity,engagement,familiarity,like_dislike,positivity,tension'.split(',') # Excluding concentration for now
#     })
  raw_data_schema.update({
      colname : dataset_schema.ColumnSchema(tf.bool, [], dataset_schema.FixedColumnRepresentation())
                   for colname in 'hallelujah_reaction,hearing_impairments'.split(',')
    })
  raw_data_metadata = dataset_metadata.DatasetMetadata(dataset_schema.Schema(raw_data_schema))
  
  # run Beam  
  with beam.Pipeline(RUNNER, options=opts) as p:
    with beam_impl.Context(temp_dir=os.path.join(OUTPUT_DIR, 'tmp')):
      # save the raw data metadata
      _ = (raw_data_metadata
        | 'WriteInputMetadata' >> tft_beam_io.WriteMetadata(
            os.path.join(OUTPUT_DIR, 'metadata/rawdata_metadata'),
            pipeline=p))
      
      # analyze and transform training
      this_query = create_query(1, EVERY_N)
      print(this_query)
      raw_data = (p 
        | 'train_read' >> beam.io.Read(beam.io.BigQuerySource(query=this_query, use_standard_sql=True))
        | 'train_filter' >> beam.Filter(is_valid))

      raw_dataset = (raw_data, raw_data_metadata)
      transformed_dataset, transform_fn = (
          raw_dataset | beam_impl.AnalyzeAndTransformDataset(preprocess_tft))
      transformed_data, transformed_metadata = transformed_dataset
      _ = transformed_data | 'WriteTrainData' >> tfrecordio.WriteToTFRecord(
          os.path.join(OUTPUT_DIR, 'train'),
          file_name_suffix='.gz',
          coder=example_proto_coder.ExampleProtoCoder(
              transformed_metadata.schema))
      
      # transform eval data
      raw_test_data = (p 
        | 'eval_read' >> beam.io.Read(beam.io.BigQuerySource(query=create_query(2, EVERY_N), use_standard_sql=True))
        | 'eval_filter' >> beam.Filter(is_valid))
      
      raw_test_dataset = (raw_test_data, raw_data_metadata)
      transformed_test_dataset = (
          (raw_test_dataset, transform_fn) | beam_impl.TransformDataset())
      transformed_test_data, _ = transformed_test_dataset
      _ = transformed_test_data | 'WriteTestData' >> tfrecordio.WriteToTFRecord(
          os.path.join(OUTPUT_DIR, 'eval'),
          file_name_suffix='.gz',
          coder=example_proto_coder.ExampleProtoCoder(
              transformed_metadata.schema))
      _ = (transform_fn
           | 'WriteTransformFn' >>
           transform_fn_io.WriteTransformFn(os.path.join(OUTPUT_DIR, 'metadata')))

preprocess(in_test_mode=False)

Launching Dataflow job hallelujah-effect-features-180613-164159 ... hang on

SELECT *
FROM
  `eim-muse.hallelujah_effect.full_hallelujah_trials_cleaned`
   WHERE MOD(FARM_FINGERPRINT(id), 10) < 7
{'tension': <tf.Tensor 'inputs/tension_copy:0' shape=(?,) dtype=float32>, 'like_dislike': <tf.Tensor 'inputs/like_dislike_copy:0' shape=(?,) dtype=float32>, 'language': <tf.Tensor 'inputs/language_copy:0' shape=(?,) dtype=string>, 'hallelujah_reaction': <tf.Tensor 'inputs/hallelujah_reaction_copy:0' shape=(?,) dtype=bool>, 'positivity': <tf.Tensor 'inputs/positivity_copy:0' shape=(?,) dtype=float32>, 'engagement': <tf.Tensor 'inputs/engagement_copy:0' shape=(?,) dtype=float32>, 'familiarity': <tf.Tensor 'inputs/familiarity_copy:0' shape=(?,) dtype=float32>, 'sex': <tf.Tensor 'inputs/sex_copy:0' shape=(?,) dtype=string>, 'location': <tf.Tensor 'inputs/location_copy:0' shape=(?,) dtype=string>, 'activity': <tf.Tensor 'inputs/activity_copy:0' shape=(?,) dtype=float32>, 'nationality': <tf.Tensor '

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: gs://eim-muse/analysis/hallelujah-effect/preproc_tft/tmp/tftransform_tmp/41021dc6b97b484488f762643b710cc2/saved_model.pb


...

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: gs://eim-muse/analysis/hallelujah-effect/preproc_tft/tmp/tftransform_tmp/5a48534a791e42cf847114bbfb9652cb/saved_model.pb


INFO:tensorflow:SavedModel written to: gs://eim-muse/analysis/hallelujah-effect/preproc_tft/tmp/tftransform_tmp/5a48534a791e42cf847114bbfb9652cb/saved_model.pb
/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:286: DeprecationWarning: Interpreting naive datetime as local 2018-06-13 16:41:59.796012. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [66]:
%bash
# ls -l preproc_tft
# ls preproc_tft/metadata
gsutil ls -l gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/
gsutil ls gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/metadata

         0  2018-06-13T16:42:55Z  gs://eim-muse/analysis/hallelujah-effect/preproc_tft/
      1139  2018-06-13T16:53:00Z  gs://eim-muse/analysis/hallelujah-effect/preproc_tft/eval-00000-of-00001.gz
      1733  2018-06-13T16:50:56Z  gs://eim-muse/analysis/hallelujah-effect/preproc_tft/train-00000-of-00002.gz
      4740  2018-06-13T16:50:56Z  gs://eim-muse/analysis/hallelujah-effect/preproc_tft/train-00001-of-00002.gz
                                 gs://eim-muse/analysis/hallelujah-effect/preproc_tft/metadata/
                                 gs://eim-muse/analysis/hallelujah-effect/preproc_tft/tmp/
TOTAL: 4 objects, 7612 bytes (7.43 KiB)
gs://eim-muse/analysis/hallelujah-effect/preproc_tft/metadata/
gs://eim-muse/analysis/hallelujah-effect/preproc_tft/metadata/rawdata_metadata/
gs://eim-muse/analysis/hallelujah-effect/preproc_tft/metadata/transform_fn/
gs://eim-muse/analysis/hallelujah-effect/preproc_tft/metadata/transformed_metadata/


/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:286: DeprecationWarning: Interpreting naive datetime as local 2018-06-13 17:13:10.653947. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


<h2> Train off preprocessed data </h2>

In [ ]:
MODEL_NAME = 'layers_16_16_16'
os.environ['MODEL_NAME'] = MODEL_NAME

In [102]:
%bash
rm -rf ${PWD}/models/${MODEL_NAME}
export PYTHONPATH=${PYTHONPATH}:$PWD/taxifare_tft
python -m trainer.task \
   --train_data_paths="gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/train*" \
   --eval_data_paths="gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/eval*"  \
   --train_batch_size=128 \
   --output_dir=${PWD}/models/${MODEL_NAME} \
   --train_steps=50000 --eval_steps=1 --job-dir=/tmp \
   --metadata_path=gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/metadata
   --hidden_units="16 16 16"

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 5, '_session_config': None, '_keep_checkpoint_max': 10, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f183382ac90>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/content/datalab/notebooks/eim-analysis/Hallelujah_Effect/models/hallelujah-effect_trained/', '_global_id_in_cluster': 0, '_save_summary_s

In [104]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://eim-muse/analysis/hallelujah-effect/models')

TensorBoard was started successfully with pid 24203. Click here to access it.

24203

/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:286: DeprecationWarning: Interpreting naive datetime as local 2018-06-13 19:49:49.305041. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [103]:
TensorBoard.stop(20767)

/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:286: DeprecationWarning: Interpreting naive datetime as local 2018-06-13 19:49:28.004080. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [108]:
%bash
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   -- \
   --train_data_paths="gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/train*" \
   --eval_data_paths="gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/eval*" \
   --train_steps=1000 \
   --train_batch_size=10 \
   --eval_steps=100 \
   --output_dir=${PWD}/models/local-ml

# %%bash
# OUTDIR=gs://${BUCKET}/analysis/hallelujah-effect/models/hallelujah-effect_trained
# JOBNAME=hallelujah_effect$(date -u +%y%m%d_%H%M%S)
# echo $OUTDIR $REGION $JOBNAME
# gsutil -m rm -rf $OUTDIR
# gcloud ml-engine jobs submit training $JOBNAME \
#    --region=$REGION \
#    --package-path=${PWD}/taxifare/trainer \
#    --module-name=trainer.task \
#    --job-dir=$OUTDIR \
#    --scale-tier=STANDARD_1 \
#    --runtime-version=1.4 \
#    -- \
#    --train_data_paths="gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/train*" \
#    --eval_data_paths="gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/eval*" \
#    --output_dir=$OUTDIR \
#    --train_steps=1000
#    --train_batch_size=10 \
#    --eval_steps=100
#    --config=hyperparam.yaml \

# --staging-bucket=gs://eim-muse-staging \

# export PYTHONPATH=${PYTHONPATH}:$PWD/taxifare_tft
# python -m trainer.task \
#    --train_data_paths="gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/train*" \
#    --eval_data_paths="gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/eval*"  \
#    --train_batch_size=10 \
#    --output_dir="gs://${BUCKET}/analysis/hallelujah-effect/models/hallelujah-effect_trained" \
#    --train_steps=5000 --eval_steps=1 --job-dir=/tmp \
#    --metadata_path=gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/metadata

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_paths=gs://eim-muse/analysis/hallelujah-effect/preproc_tft/train*', u'--eval_data_paths=gs://eim-muse/analysis/hallelujah-effect/preproc_tft/eval*', u'--train_steps=1000', u'--train_batch_size=10', u'--eval_steps=100', u'--output_dir=/content/datalab/notebooks/eim-analysis/Hallelujah_Effect/models/local-ml'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '

In [105]:
TensorBoard().start('./models/local-ml')

TensorBoard was started successfully with pid 24756. Click here to access it.

24756

/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:286: DeprecationWarning: Interpreting naive datetime as local 2018-06-13 20:00:58.941045. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [92]:
TensorBoard().stop(11815)

/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:286: DeprecationWarning: Interpreting naive datetime as local 2018-06-13 18:39:39.500024. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [69]:
%bash
gsutil ls -l gs://${BUCKET}/analysis/hallelujah-effect/preproc_tft/

         0  2018-06-13T16:42:55Z  gs://eim-muse/analysis/hallelujah-effect/preproc_tft/
      1139  2018-06-13T16:53:00Z  gs://eim-muse/analysis/hallelujah-effect/preproc_tft/eval-00000-of-00001.gz
      1733  2018-06-13T16:50:56Z  gs://eim-muse/analysis/hallelujah-effect/preproc_tft/train-00000-of-00002.gz
      4740  2018-06-13T16:50:56Z  gs://eim-muse/analysis/hallelujah-effect/preproc_tft/train-00001-of-00002.gz
                                 gs://eim-muse/analysis/hallelujah-effect/preproc_tft/metadata/
                                 gs://eim-muse/analysis/hallelujah-effect/preproc_tft/tmp/
TOTAL: 4 objects, 7612 bytes (7.43 KiB)


/usr/local/envs/py2env/lib/python2.7/site-packages/simplejson/encoder.py:286: DeprecationWarning: Interpreting naive datetime as local 2018-06-13 17:13:54.492991. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [11]:
%writefile /tmp/test.json
{"age":"29.0","activity":3.0}

Writing /tmp/test.json


In [13]:
%bash
model_dir=$(ls $PWD/hallelujah-effect_trained/export/exporter/)
gcloud ml-engine local predict \
    --model-dir=./hallelujah-effect_trained/export/exporter/${model_dir} \
    --json-instances=/tmp/test.json

CLASS_IDS  CLASSES  LOGISTIC               LOGITS                 PROBABILITIES
[0]        [u'0']   [0.45960211753845215]  [-0.1619444638490677]  [0.5403978824615479, 0.45960214734077454]


  from ._conv import register_converters as _register_converters
2018-06-13 14:31:44.643762: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA



# To Do

- LASSO to identify important features
- Hyperparameter search
- More plots and statistics from the dataset with which I'm working here
- Bring in rows with missing values
- Feature engineering (physiological signals, MIR, feature crosses, variable-width binning)
- Include signals with good quality only in reaction range
- Customize estimator to add additional metrics